<a href="https://colab.research.google.com/github/Kimjibeom/LSTM/blob/main/LSTM_%EA%B5%90%ED%86%B5%EB%9F%89_%EC%98%88%EC%B8%A1(base).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive 마운트 하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 경로 설정
import os
os.chdir('/content/drive/My Drive/')

# 라이브러리

In [1]:
from keras import backend
from pandas import DataFrame
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import model_from_json
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('.csv')
df.head()

# 스케일링

In [ ]:
def minmax_scaler(X, minmax_range):
  scaled = (X - minmax_range[0]) / (minmax_range[1] - minmax_range[0])
  return scaled

look_back 인자는 얼마만큼의 이전 수치를 데이터로 만들것인가 결정

In [ ]:
# 192는 수정필요
def create_dataset(data, data2, look_back=192):
  dataX, dataY = [], []
  for i in range(len(data)-2*look_back):
    dataX.append(data[i:(i+look_back)])
    dataY.append(data2[i+2*(look_back)])
  return np.array(dataX), np.array(dataY)

In [ ]:
df = df.interpolate(method='linear', limint_direction='both', axis=0)

train_set = df.iloc[:int(len(df)*0.8)]
test_set = df.iloc[int(len(df)*0.8):]

print(f'전체 데이터: {len(df)}')
print(f'학습 데이터: {len(train_set)}')
print(f'시험 데이터: {len(test_set)}')

In [ ]:
# 점유율
pw = np.array(train_set['점유율'])
pw = pw.reshape(pw.shape[0],1)
# 도로총교통량
tf = np.array(train_set['도로총교통량'])
tf = tf.reshape(tf.shape[0],1)
# 평균통행속도
spd = np.array(train_set['평균통행속도'])
spd = spd.reshape(spd.shape[0],1)

pw = minmax_scaler(pw, [min(pw), max(pw)])
tf = minmax_scaler(tf, [0,6000])
spd = minmax_scaler(spd, [0,200])

total = np.hstack([pw,tf,spd])
X_train, y_train = create_dataset(total,pw)

print(X_train.shape)
print(y_train.shape)

# 모델

In [ ]:
model = Sequential()
model.add(LSTM(32,input_shape=(X_train.shape[1],4),return_sequences=True))
model.add(LSTM(32,input_shape=(X_train.shape[1],4)))
# model.add(TimeDistributed(Dense(1)))
model.add(Dense(1))

model.compile(loss='mae', optimizer='adam')
model.fit(X_train, y_train, epochs=50, batch_size=384, verbose=2, shuffle=False)

from keras.models import model_from_json

model_json = model.to_json()
jsonname = 'lstm_model.json'
h5name = 'lstm_model.h5'
with open(jsonname,"w") as json_file:
    json_file.write(model_json)

model.save_weights(h5name)
print("Saved model to disk")
print('finished!')

# 테스트

In [ ]:
def inverse_scaler(X,minmax_range):
    inv = X * (minmax_range[1]-minmax_range[0]) + minmax_range[0]
    return inv

In [ ]:
pw_range = [min(df['점유율']), max(df['점유율'])]
# 점유율
pw = np.array(train_set['점유율'])
pw = pw.reshape(pw.shape[0],1)
# 도로총교통량
tf = np.array(train_set['도로총교통량'])
tf = tf.reshape(tf.shape[0],1)
# 평균통행속도
spd = np.array(train_set['평균통행속도'])
spd = spd.reshape(spd.shape[0],1)

pw = minmax_scaler(pw, pw_range)
tf = minmax_scaler(tf, [0,6000])
spd = minmax_scaler(spd, [0,200])

Test = np.hstack([pw,tf,spd])

json_file = open('lstm_model.json',"r")
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights('lstm_model.h5')
model.compile(loss='mse', optimizer='adam')

predictions = []
y_test = []

i = 0; cnt = 0
while (i < len(Test)-384):
    X_test, test_y = create_dataset(Test[i:i+385],pw[i:i+385],192)
    pred = model.predict(X_test)
#     print(pred, end=' ')
    predictions.append(pred)
    y_test.append(test_y)

    i += 1
    cnt += 1
##### inverse transform ##############
y_test = inverse_scaler(np.array(y_test),pw_range)
predictions = inverse_scaler(np.array(predictions),pw_range)
y_test = np.vstack(y_test).squeeze()
predictions = np.vstack(predictions).squeeze()